In [4]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
'''Used to help format and organize the messy web by fixing bad HTML and presenting us with
easily-traversible Python objects repreenting XML structures.'''
import pandas as pd
import tweepy
from tweepy import OAuthHandler
import json
from unidecode import unidecode
import pprint
pp = pprint.PrettyPrinter(indent=4)
import warnings
warnings.filterwarnings("ignore")

In [34]:
class TwitterClient(object):
    def __init__(self):
        #Twitter Credentials
        with open('twitter_cred.json') as secret_info:
            info = json.load(secret_info)
            consumer_key = info['CONSUMER_KEY']
            consumer_secret = info['CONSUMER_SECRET']
            access_key = info['ACCESS_KEY']
            access_secret = info['ACCESS_SECRET']
            
            try:

                # Creating the API endpoint
                auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
                #setting the access token and secret
                auth.set_access_token(access_key,access_secret)
                self.api = tweepy.API(auth,wait_on_rate_limit=True)
            except tweepy.TweepError as e:
                print(f"Error: Twitter Authentication Failed - \n{str(e)}")
    
    #Function used to fetch tweets
    def get_tweets(self,query,maxTweets = 1000):
        tweets = []
        sinceId = []
        max_id = -1
        tweetCount = 0
        tweetsPerQry = 100
        date_since = "2019-01-11"
        
        while tweetCount < maxTweets:
            try:
#                 if (max_id <= 0):
#                     if (not sinceId):
#                         new_tweets = self.api.search(q=query,count=tweetsPerQry)
#                     else:
#                         new_tweets = self.api.search(q=query,count=tweetsPerQry,since_id = sinceId)
#                 else:
#                     if (not sinceId):
#                         new_tweets = self.api.search(q=query,count=tweetsPerQry,max_id=str(max_id-1))
#                     else:
#                         new_tweets = self.api.search(q=query,count=tweetsPerQry,max_id=str(max_id-1),since_id=sinceId)
                new_tweets = tweepy.Cursor(self.api.search,
                   q=query,
                  lang='en',
                  since=date_since).items(tweetsPerQry)
                if not new_tweets:
                    print("No more tweets found")
                    break
                
                for tweet in new_tweets:
                    parsed_tweet = {}
                    parsed_tweet['tweets'] = tweet.text
                    
                    #Appending parsed tweets to tweets list
                    if tweet.retweet_count > 0:
                        #if the tweet has retweets, ensure that it is appended only once
                        if parsed_tweet and not 'RT @' not in tweets:
                            tweets.append(parsed_tweet)
                    else:
                        tweets.append(parsed_tweet)
#                 tweetCount += len(new_tweets)
#                 print("Downloaded {0} tweets".format(tweetCount))
#                 max_id = new_tweets[-1].id
            except tweepy.TweepError as e:
                print("Tweepy error:" + str(e))
                break
        t = pd.DataFrame(tweets)
        t.columns = ['List of Tweets']
        with open('tweets_pull5.csv','a+') as f:
            t.to_csv(f,header=['List of Tweets'])
        return t
    
    def remove_pattern(text,pattern_regex):
        r = re.findall(pattern_regex,text)
        for i in r:
            text = re.sub(i,'',text)
        return text
    
                        
                        
                
        
        

In [35]:
selenium_client = TwitterClient()
# l = ['#algaebloom', '#chemicalspill', '#drinkingwatercontamination', '#drought', '#drywell', '#ecoli', '#flashflood', '#groundwatercontamination', '#industrialpollution', '#oilspill', '#pipeleak', '#wastewater', '#waterandsanitation', '#watercontamination', '#watermanagement', '#waterpollution', 'algae bloom', 'beach closure', 'chemical spill', 'coal ash water', 'drinking water', 'drinking water contamination', 'drought', 'dry well', 'e-coli', 'ecoli', 'flash flood', 'flood warning', 'groundwater contamination', 'groundwater pollution', 'industrial pollution', 'industrial solvents', 'lake contamination', 'lake pollution', 'lead water', 'marine plastic pollution', 'metal hydroxide', 'oil spill', 'oxygen depletion', 'pipe leak', 'point source pollution', 'river contamination', 'stream contamination', 'surface water contamination', 'toxic waste water', 'untreated waste', 'waste spill', 'waste water', 'water and sanitation', 'water bacteria', 'water contamination', 'water impurity', 'water management', 'water plastic', 'water pollution', 'water sewage', 'waterbourne disease','hurricane', 'sandy', 'storm', 'survivor', 'effort', 'u.s', 'east', 'flood', 'path', 'map','sound', 'metal', 'backyard', 'recording', 'hear', 'music', 'audio', 'cicada', 'scrap', 'temperature','climate', 'change', 'talk', 'week', 'need', 'say', 'study', 'ice', 'action', 'carbon','new', 'specie', 'york', 'farmer', 'discover', 'photo', 'energy', 'normal', 'animal', 'nyc','sea', 'rise', 'level', 'video', 'study', 'fall', 'photo', 'underwater', 'deep', 'melting','day', 'earth', 'mother', 'world', 'gift', 'photo', 'fall', 'green', 'celebrate', 'endanger','baby', 'photo', 'animal', 'week', 'panda', 'picture', 'zoo', 'elephant', 'gorilla', 'giraffes','animal', 'week', 'photo', 'picture', 'tiger', 'elephant', 'monkey', 'panda', 'lion', 'cub','use', 'park', 'photo', 'national', 'amazing', 'animal', 'odd', 'weather.com', 'way','time', 'celebrate', 'lapse', 'year', 'video', 'week', 'cool', 'wildlife', 'puppy', 'draw','keystone', 'xl', 'pipeline', 'sand', 'tar', 'obama', 'protester', 'arrest', 'student', 'department','extreme', 'weather', 'week', 'photo', 'hero', 'hill', 'hilarious', 'highlight', 'high', 'zoonose','photo', 'animal', 'life', 'nature', 'thing', 'pollution', 'world', 'big', 'storm', 'free','spill', 'bp', 'oil', 'trial', 'gulf', 'exxon', 'molasse', 'testify', 'halliburton', 'year','home', 'save', 'tip', 'money', 'orca', 'energy', 'rescue', 'winterize', 'food', 'abandon','public', 'kill', 'lion', 'preserve', 'transit', 'zoo', 'explorer', 'slaughter', 'cub', 'bear','project', 'reuse', 'diy', 'idea', 'old', 'shirt', 'craft', 'bag', 'make','green', 'eco', 'electric', 'friendly', 'tip', 'labor', 'burial', 'power', 'event', 'new','video', 'capture', 'make', 'camera', 'thing', 'pet', 'animal', 'puppy', 'kitten', 'panda','dog', 'cat', 'photo', 'video', 'pet', 'adorable', 'love', 'owner', 'rescue', 'head','say', 'water', 'study', 'food', 'global', 'warming', 'scientist', 'ban', 'new', 'highlight','oil', 'gas', 'california', 'future', 'natural', 'industry', 'fracke', 'plan', 'platform', 'drought','u.s', 'winter', 'fukushima', 'state', 'storm', 'hit', 'coast', 'american', 'outlook', 'hurricane']
l = ['#algaebloom', '#chemicalspill', '#drinkingwatercontamination']

l=" OR ".join(l)
search_terms = l
new_search = search_terms + "-filter:retweets"
print(new_search)
tweets_df = selenium_client.get_tweets(new_search)
tweets_df

#algaebloom OR #chemicalspill OR #drinkingwatercontamination-filter:retweets


KeyboardInterrupt: 